In [6]:
pwd


'C:\\Users\\12345\\Documents\\בינה מלכותית\\EX3'

In [52]:
import pandas as pd

In [53]:
df_rating = pd.read_csv("ratings.csv")

In [54]:
df_rating.shape

(454517, 3)

In [55]:
df_rating.head()

,user_id,book_id,rating
0,511,277,4
1,2766,54,5
2,2290,2499,2
3,3920,1603,5
4,2457,4,5


In [56]:
df_users = pd.read_csv("users.csv")
df_users.head()

,user_id,location,age
0,1,North Dakota,69
1,2,California,24
2,4,Missouri,98
3,6,Rhode Island,66
4,8,Hawaii,64


In [57]:
df_rating_users = pd.merge(df_users, df_rating, how="inner", on=["user_id"])
df_rating_users.head()

,user_id,location,age,book_id,rating
0,1,North Dakota,69,1187,5
1,1,North Dakota,69,171,3
2,1,North Dakota,69,162,4
3,1,North Dakota,69,2133,5
4,1,North Dakota,69,219,4


In [99]:
df_agg = df_rating.groupby('book_id').rating.agg(['mean','count'])
df_agg.reset_index(inplace=True)
df_agg = df_agg.rename(columns={'mean': 'vote_average', 'count': 'vote_count'}) 
df_agg.head()

,book_id,vote_average,vote_count
0,1,4.238662,1764
1,2,4.058140,1634
2,3,3.376528,1227
3,4,4.364625,1747
4,5,3.748036,1655


In [61]:
(df_rating.book_id == 1).sum()

1764

In [62]:
df_rating.user_id.unique().shape

(5000,)

In [63]:
df_rating.book_id.unique().shape

(5000,)

In [64]:
metadata =  pd.read_excel("books.xlsx")
metadata.title.fillna(metadata.original_title, inplace=True)
metadata.original_title.fillna(metadata.title, inplace=True)
metadata.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780440e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrֳƒֲƒֳ‚ֲ©",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780320e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780060e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780740e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [65]:
sorted(df_rating.book_id.unique())[0],sorted(df_rating.book_id.unique())[-1]

(1, 5060)

In [66]:
sorted(df_rating.user_id.unique())[0],sorted(df_rating.user_id.unique())[-1]


(1, 5000)

In [67]:
def calculate_recomendation(metadata,df,k):
    C = df['vote_average'].mean()
    m = df['vote_count'].quantile(0.90)
    me = pd.merge(metadata, df, how="inner", on=["book_id"])
    q_books = me[me['vote_count'] >= m]
    # Function that computes the weighted rating of each movie
    def weighted_rating(x, m=m, C=C):
        v = x['vote_count']
        R = x['vote_average']
        # Calculation based on the IMDB formula
        return (v/(v+m) * R) + (m/(m+v) * C)
    # Define a new feature 'score' and calculate its value with `weighted_rating()`
    q_books['score'] = q_books.apply(weighted_rating, axis=1)  
    #Sort movies based on score calculated above
    q_books = q_books.sort_values('score', ascending=False)
    return q_books[['book_id','original_title', 'score']].head(k)


In [68]:
calculate_recomendation(metadata,df_agg,10)

<ipython-input-67-32b4097a1303>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_books['score'] = q_books.apply(weighted_rating, axis=1)


,book_id,original_title,score
24,25,Harry Potter and the Deathly Hallows,4.338028
3,4,To Kill a Mockingbird,4.299843
101,102,Where the Wild Things Are,4.273212
84,85,The Giving Tree,4.240309
49,50,Where the Sidewalk Ends: The Poems and Drawing...,4.239724
30,31,The Help,4.238851
143,144,"Unbroken: A World War II Story of Survival, Re...",4.221864
26,27,Harry Potter and the Half-Blood Prince,4.213906
0,1,The Hunger Games,4.187383
132,133,Anne of Green Gables,4.181489


In [21]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
#q_books['score'] = q_books.apply(weighted_rating, axis=1)

In [69]:
def get_simply_recommendation(k):
    return calculate_recomendation(metadata,df_agg,k)

In [70]:
get_simply_recommendation(10)

<ipython-input-67-32b4097a1303>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_books['score'] = q_books.apply(weighted_rating, axis=1)


,book_id,original_title,score
24,25,Harry Potter and the Deathly Hallows,4.338028
3,4,To Kill a Mockingbird,4.299843
101,102,Where the Wild Things Are,4.273212
84,85,The Giving Tree,4.240309
49,50,Where the Sidewalk Ends: The Poems and Drawing...,4.239724
30,31,The Help,4.238851
143,144,"Unbroken: A World War II Story of Survival, Re...",4.221864
26,27,Harry Potter and the Half-Blood Prince,4.213906
0,1,The Hunger Games,4.187383
132,133,Anne of Green Gables,4.181489


In [51]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# #plot
# plt.figure(figsize=(16,6))

# ax = sns.barplot(x=q_books['score'].head(10), y=q_books['original_title'].head(10), data=q_books, palette='deep')

# plt.xlim(3.5, 4.5)
# plt.title('"Best" BOOKS by TMDB Votes', weight='bold')
# plt.xlabel('Weighted Average Score', weight='bold')
# plt.ylabel('Book Title', weight='bold')

In [71]:
def get_simply_recommendation(place , k):
    a = df_rating_users [df_rating_users ['location']== place]
    df_agg = a.groupby('book_id').rating.agg(['mean','count'])
    df_agg.reset_index(inplace=True)
    df_agg = df_agg.rename(columns={'mean': 'vote_average', 'count': 'vote_count'}) 
    
    return calculate_recomendation(metadata,df_agg,k)

In [72]:
get_simply_recommendation('Ohio' , 10)

<ipython-input-67-32b4097a1303>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_books['score'] = q_books.apply(weighted_rating, axis=1)


,book_id,original_title,score
125,126,Dune,4.367963
142,143,All the Light We Cannot See,4.317786
143,144,"Unbroken: A World War II Story of Survival, Re...",4.266087
23,24,Harry Potter and the Goblet of Fire,4.249728
101,102,Where the Wild Things Are,4.226877
461,490,Maus: A Survivor's Tale : My Father Bleeds His...,4.213664
1219,1462,The Orphan Master's Son,4.213664
876,983,Between the World and Me,4.213664
118,119,The Handmaid's Tale,4.199565
88,89,The Princess Bride,4.190062


In [100]:
def get_simply_age_recommendation(age , k):
#     a = df_rating_users[df_rating_users ['age'] >= ((age/10)*10 + 1) and df_rating_users ['age'] <= ((age/10 + 1)*10)]
    a = df_rating_users.loc[(df_rating_users ['age'] >= ((age/10)*10 + 1)) & (df_rating_users ['age'] <= ((age/10 + 1)*10))]

    df_agg = a.groupby('book_id').rating.agg(['mean','count'])
    df_agg.reset_index(inplace=True)
    df_agg = df_agg.rename(columns={'mean': 'vote_average', 'count': 'vote_count'}) 
    
    return calculate_recomendation(metadata,df_agg,k)

In [101]:
get_simply_age_recommendation(28,10)

<ipython-input-67-32b4097a1303>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_books['score'] = q_books.apply(weighted_rating, axis=1)


,book_id,original_title,score
101,102,Where the Wild Things Are,4.310083
24,25,Harry Potter and the Deathly Hallows,4.303093
84,85,The Giving Tree,4.293723
3,4,To Kill a Mockingbird,4.289958
49,50,Where the Sidewalk Ends: The Poems and Drawing...,4.252111
143,144,"Unbroken: A World War II Story of Survival, Re...",4.228776
30,31,The Help,4.186088
26,27,Harry Potter and the Half-Blood Prince,4.186088
266,267,The Nightingale,4.172023
0,1,The Hunger Games,4.163516


PART 2

# Colaborative  Filtering

In [88]:
import numpy as np

In [105]:
df_rating.head()

,user_id,book_id,rating
0,511,277,4
1,2766,54,5
2,2290,2499,2
3,3920,1603,5
4,2457,4,5


In [145]:
from sklearn.metrics.pairwise import pairwise_distances

def build_CF_prediction_matrix(sim):
    df_Rowuser_ColBook = df_rating.pivot(index='user_id',columns='book_id',values='rating')

    ratings = df_Rowuser_ColBook.to_numpy()
    mean_user_rating = np.nanmean(ratings,axis=1,keepdims = True)

    #הורדה של ממוצע הצבעות לכל משתמש
    ratings_diff = (ratings - mean_user_rating)
    ratings_diff[np.isnan(ratings_diff)]=0
    ratings_diff.round(2)
    
    user_similarity = 1-pairwise_distances(ratings_diff, metric=sim)
    print(user_similarity.shape)
    pd.DataFrame(user_similarity).round(2)
    pred = mean_user_rating + user_similarity.dot(ratings_diff) / np.abs(user_similarity).sum(axis=1,keepdims=True)
    return pred

In [148]:
def get_CF_recommendation(user_id,k):
    #boolean indexing
    #להכניס נן לספרשהואכבר קרא
    pred = build_CF_prediction_matrix('cosine')
    pred[~np.isnan(ratings)]=np.nan
    pred_pd = pd.DataFrame(pred , columns = df_Rowuser_ColBook.columns)
    
    #חזיר את רשימת הספרים הממולצים לא כולל מה שקרא בעצמו
    #מסדר את השורה של כל יוזר לפי הציון(וליו) ואזלוקח את האינדקס שזה הספר
    user_rec=pred_pd.loc[4].sort_values(ascending = False).index[:10]
    return metadata.set_index('book_id').loc[user_rec, 'title']

In [152]:
get_CF_recommendation(20,10)

(5000, 5000)


book_id
1      The Hunger Games (The Hunger Games, #1)
143                All the Light We Cannot See
31                                    The Help
250                                     Wonder
17        Catching Fire (The Hunger Games, #2)
4                        To Kill a Mockingbird
146                              The Goldfinch
267                            The Nightingale
291                          Cutting for Stone
225                               East of Eden
Name: title, dtype: object

In [147]:
build_CF_prediction_matrix('cosine')

(5000, 5000)


array([[3.68023517, 3.64953318, 3.55293793, ..., 3.60349107, 3.60538499,
        3.60664203],
       [4.47692204, 4.72321656, 4.30417573, ..., 4.35988827, 4.35851025,
        4.35996414],
       [1.91008199, 1.90604477, 1.85044194, ..., 1.86592517, 1.86425898,
        1.86556079],
       ...,
       [4.35178965, 4.32373572, 4.35315963, ..., 4.32943237, 4.32765218,
        4.32773439],
       [4.19654323, 4.14622987, 4.09210832, ..., 4.11416208, 4.1162666 ,
        4.11636131],
       [4.03552559, 3.63622012, 3.35695638, ..., 3.53234043, 3.56104276,
        3.55942448]])

In [119]:
df_Rowuser_ColBook = df_rating.pivot(index='user_id',columns='book_id',values='rating')

ratings = df_Rowuser_ColBook.to_numpy()
mean_user_rating = np.nanmean(ratings,axis=1,keepdims = True)

#הורדה של ממוצע הצבעות לכל משתמש
ratings_diff = (ratings - mean_user_rating)
ratings_diff[np.isnan(ratings_diff)]=0
ratings_diff.round(2)


array([[ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.64,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.12,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 1.44,  0.  , -0.56, ..., -1.56,  0.  ,  0.  ]])

In [118]:
np.unique(ratings)
ratings[~np.isnan(ratings)]


array([5., 4., 5., ..., 3., 4., 2.])

In [ ]:
#brodcasting -- אין מטריצה פחות וקטור ולכן 
# NUMPY יודע לעשות פעולות חיבור וחיסור וכפל וכו על מערכים
# אם זה לא באותו גודל 
# אם במימיד אחד הם מתאימים בדיוק ובמימיד השני חסר הוא ישדר את המימד לכל המטריצה 

In [120]:
from sklearn.metrics.pairwise import pairwise_distances


user_similarity = 1-pairwise_distances(ratings_diff, metric='cosine')
print(user_similarity.shape)
pd.DataFrame(user_similarity).round(2)

(5000, 5000)


,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,1.00,-0.00,0.02,0.10,0.00,-0.00,0.01,0.01,-0.06,-0.01,...,0.01,0.02,0.02,-0.03,0.01,0.06,-0.01,-0.01,0.08,0.00
1,-0.00,1.00,-0.01,0.11,0.00,0.02,0.00,0.00,0.05,0.01,...,0.00,0.00,0.06,0.03,0.11,0.00,-0.02,0.00,0.03,0.00
2,0.02,-0.01,1.00,-0.01,0.00,0.02,-0.02,0.04,-0.03,0.02,...,0.02,-0.00,-0.00,-0.01,0.01,0.01,0.02,0.00,-0.02,0.00
3,0.10,0.11,-0.01,1.00,0.00,-0.01,0.01,0.01,-0.00,0.06,...,0.02,-0.03,0.05,-0.00,0.10,0.09,0.03,-0.01,0.06,0.00
4,0.00,0.00,0.00,0.00,1.00,0.00,0.00,-0.01,0.01,-0.01,...,-0.02,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.06,0.00,0.01,0.09,0.00,-0.01,-0.02,-0.01,-0.01,0.01,...,0.02,0.02,0.03,0.01,-0.06,1.00,-0.02,0.00,0.04,-0.01
4996,-0.01,-0.02,0.02,0.03,0.00,-0.02,-0.03,-0.01,0.01,0.05,...,0.01,-0.00,0.05,0.00,0.02,-0.02,1.00,-0.04,0.07,0.00
4997,-0.01,0.00,0.00,-0.01,0.01,0.00,0.01,0.00,0.00,0.00,...,0.00,-0.01,0.00,0.00,-0.00,0.00,-0.04,1.00,0.00,-0.02
4998,0.08,0.03,-0.02,0.06,0.00,0.01,0.03,-0.08,0.06,0.07,...,-0.04,0.05,0.13,0.04,0.03,0.04,0.07,0.00,1.00,0.00


In [122]:
#pred the book foreach user
#ממוצע משוכלל לכל ספר של היוזרים שדומים ל משוכללים במדד הדימיוןי
#user_similarity.dot(ratings_diff)
pred = mean_user_rating + user_similarity.dot(ratings_diff) / np.abs(user_similarity).sum(axis=1,keepdims=True)

In [127]:
df_users.shape

(5000, 3)

In [135]:
#boolean indexing
#להכניס נן לספרשהואכבר קרא
pred[~np.isnan(ratings)]=np.nan
pred_pd = pd.DataFrame(pred , columns = df_Rowuser_ColBook.columns)
pred_pd.head()
pred_pd.shape
ratings.shape

(5000, 5000)

In [136]:
#חזיר את רשימת הספרים הממולצים לא כולל מה שקרא בעצמו
#מסדר את השורה של כל יוזר לפי הציון(וליו) ואזלוקח את האינדקס שזה הספר
user_rec = pred_pd.apply(lambda row: row.sort_values(ascending = False).index[:10] ,axis = 1)

In [138]:
user_rec[4]

Int64Index([1, 143, 31, 250, 17, 4, 146, 267, 291, 225], dtype='int64', name='book_id')

In [150]:
user_rec=pred_pd.loc[20].sort_values(ascending = False).index[:10]

In [151]:
metadata.set_index('book_id').loc[user_rec, 'title']

book_id
18    Harry Potter and the Prisoner of Azkaban (Harr...
25    Harry Potter and the Deathly Hallows (Harry Po...
1               The Hunger Games (The Hunger Games, #1)
17                 Catching Fire (The Hunger Games, #2)
31                                             The Help
10                                  Pride and Prejudice
4                                 To Kill a Mockingbird
47                                       The Book Thief
70                      Ender's Game (Ender's Saga, #1)
11                                      The Kite Runner
Name: title, dtype: object

In [139]:
def get_CF_recommendation(user_id,k):
    pred[~np.isnan(ratings)]=np.nan
    pred_pd = pd.DataFrame(pred , columns = df_Rowuser_ColBook.columns)
    #שורה מפנדס עםLOC
    user_rec = pred_pd.loc[user_id].sort_values(ascending = False).index[:10]
 
    return metadata.set_index('book_id').loc[user_rec[user_id], 'title']

In [140]:
get_CF_recommendation(4,10)

book_id
1      The Hunger Games (The Hunger Games, #1)
143                All the Light We Cannot See
31                                    The Help
250                                     Wonder
17        Catching Fire (The Hunger Games, #2)
4                        To Kill a Mockingbird
146                              The Goldfinch
267                            The Nightingale
291                          Cutting for Stone
225                               East of Eden
Name: title, dtype: object

In [ ]:
#dictionary for bookid
book_id_mapping = {j:i for i,j in enumerate (sorted(df_rating.book_id.unique()))}



In [ ]:
#matrix
from  scipy.sparse import coo_matrix

df_rating ["dic_book"] = df_rating.book_id.map(lambda x: book_id_mapping[x] )

df_rating.head()


In [ ]:
matrix = coo_matrix((df_rating.rating, (df_rating.dic_book, df_rating.user_id)))

In [ ]:
matrix.toarray()